In [5]:
from __future__ import print_function
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir, rename
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [6]:
songs_to_step = ['Anubis', 'Bend Your Mind', 'Boogie Down', 'Bouff', 'Bubble Dancer']

In [9]:
sample_rate_down = 1
hop_length_down = 8
sr = 11025 * 16 / sample_rate_down
hop_length = 512 / (sample_rate_down * hop_length_down)
samples_per_beat = 24 / 4
steps_per_bar = 24
class SongFile:
    # misc includes
    # - offset
    # - bpm
    def load_misc_from_music(self, y):
        _, beat_frames = librosa.beat.beat_track(y=y, sr=sr, hop_length=hop_length)
        beat_times = librosa.frames_to_time(beat_frames, sr=sr, hop_length=hop_length)
        self.offset = beat_times[0]
        self.bpm = get_beats(beat_times, beat_frames)

    def calculate_indices(self, y):
        # take samples_per_beat samples for each beat (need 3rds, 8ths)
        seconds = len(y) / sr
        num_samples = int(seconds * samples_per_beat * self.bpm / 60)
        beat_length = 60. / self.bpm
        sample_length = beat_length / samples_per_beat
        
        if self.offset < 0:
            self.offset += 4 * beat_length
        
        sample_times = [self.offset + (sample_length * i) for i in range(num_samples)]
        # only take samples where music still playing
        self.indices = [round(time * sr) for time in sample_times if round(time * sr) < len(y)]
        
    def calculate_features(self, y):
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        beat_frames = librosa.samples_to_frames(self.indices)

        # Compute MFCC features from the raw signal
        mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

        # And the first-order differences (delta features)
        mfcc_delta = librosa.feature.delta(mfcc)

        # Stack and synchronize between beat events
        # This time, we'll use the mean value (default) instead of median
        beat_mfcc_delta = librosa.feature.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)

        # Compute chroma features from the harmonic signal
        chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

        # Aggregate chroma features between beat events
        # We'll use the median value of each feature between beat frames
        beat_chroma = librosa.feature.sync(chromagram, beat_frames, aggregate=np.median)

        custom_hop = 256
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=custom_hop)
        onsets = librosa.onset.onset_detect(y=y, sr=sr, onset_envelope=onset_env, hop_length=custom_hop)

        i = 0
        onset_happened_in_frame = [0] * (len(self.indices) + 1)
        for onset in onsets:
            onset_scaled = onset * custom_hop
            while abs(onset_scaled - self.indices[i]) > abs(onset_scaled - self.indices[i + 1]):
                i += 1
            onset_happened_in_frame[i] = max(onset_env[onset], onset_env[onset + 1], onset_env[onset + 2], onset_env[onset + 3], onset_env[onset + 4])

        indices = [0]
        indices.extend(self.indices)
        max_offset_bounds = [(int(indices[i] / custom_hop), int(indices[i + 1] / custom_hop)) for i in range(len(indices) - 1)]
        max_offset_strengths = [max(onset_env[bounds[0]:bounds[1]]) for bounds in max_offset_bounds]
        max_offset_strengths.append(0)

        # Finally, stack all beat-synchronous features together
        self.beat_features = np.vstack([beat_chroma, beat_mfcc_delta, [onset_happened_in_frame, max_offset_strengths]])

    def __init__(self, key, folder, music_file, load_type):
        key = key
        self.folder = folder
        self.music_file = music_file
        y = None
        print ('Loading song {0}'.format(key))
        
        if load_type == 'from_music' or load_type == 'from_stepfile':
            if load_type == 'from_music':
                print ('Loading music')
                y, _ = librosa.load(self.music_file, sr=sr)
                print ('Calculating misc from music')
                self.load_misc_from_music(y)
            else:
                print ('Loading misc from stepfile')
                self.load_misc_from_stepfile()
                if self.bpm == 0:
                    raise Exception('Inconsistent bpm')
                print ('Loading music')
                y, _ = librosa.load(self.music_file, sr=sr)
                

            print ('Calculating indices')
            self.calculate_indices(y)
            print ('Calculating features')
            self.calculate_features(y)
            print ('Saving song\n')
            pd.DataFrame([self.offset, self.bpm]).to_csv('data/{0}_misc.csv'.format(key), index=False)
            pd.DataFrame(self.beat_features).to_csv('data/{0}_beat_features.csv'.format(key), index=False)
            
        if load_type == 'from_store':
            if not '{0}_beat_features.csv'.format(key) in listdir('data'):
                print ('Song hasnt been loaded yet')
            else:
                [self.offset], [self.bpm] = pd.read_csv('data/{0}_misc.csv'.format(key)).values
                self.beat_features = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values


# # Some useful functions to load induvidual or lists of songs
# - load_song(pack: String, pack: String, force_new: Bool)
# - load_songs(songs: Array(Pair(String~pack, String~title)), force_new: Bool)
# - load_all_songs(force_new: Bool)

# In[4]:

def load_songs(songs, load_type):
    return {'{0}~{1}'.format(song[0], song[1]): SongFile(song[0], song[1], load_type) for song in songs}

# # Functions to get bpm from song
# - get_beats(beat_times: Array(Float), beat_frames: Array(Int))

# In[5]:

def get_beats(beat_times, beat_frames):
    changes = []
    changes_time = []
    for i in range(len(beat_frames) - 1):
        changes.append(beat_frames[i + 1] - beat_frames[i])
        changes_time.append(beat_times[i + 1] - beat_times[i])

    sorted_changes = sorted(changes)
    median = sorted_changes[int(len(changes) / 2)]
    median = max(set(sorted_changes), key=sorted_changes.count)

    changes_counted = [False] * len(changes)
    time_changes_sum = 0
    time_changes_count = 0
    for i in range(len(changes)):
        # can use other factors (eg if song has a slow part take double beats into accout)
        # in [0.5, 1, 2]:
        for change_factor in [1]:
            if abs((changes[i] * change_factor) - median) <= hop_length_down:
                changes_counted[i] = True
                time_changes_sum += (changes_time[i] * change_factor)
                time_changes_count += change_factor
            
    average = time_changes_sum / time_changes_count
    
    time_differences = []
    earliest_proper_beat = 1
    for i in range(1, len(beat_times) - 1):
        if changes_counted[i] & changes_counted[i - 1]:
            earliest_proper_beat = i
            break
            
    last_proper_beat = len(beat_times) -2
    for i in range(1, len(beat_times) - 1):
        if changes_counted[len(beat_times) - i - 1] & changes_counted[len(beat_times) - i - 2]:
            last_proper_beat = len(beat_times) - i - 1
            break
    
    time_differences = []
    buffer = 5
    for i in range(20):
        start_beat = earliest_proper_beat + buffer * i
        if changes_counted[start_beat] & changes_counted[start_beat - 1]:
            for j in range(20):
                end_beat = last_proper_beat - buffer * j
                if changes_counted[end_beat] & changes_counted[end_beat - 1]:
                    time_differences.append(beat_times[end_beat] - beat_times[start_beat])
        
    # get num beats, round, and make new average
    new_averages = [time_difference / round(time_difference / average) for time_difference in time_differences]
    #print (new_averages)
    new_averages.sort()
    num_averages = len(new_averages)
    #new_average = sum(new_averages[5:num_averages - 5]) / (num_averages - 10)
    new_average = new_averages[int(num_averages/2)]
    bpm = 60./new_average
    while bpm >= 200:
        bpm /= 2
    while bpm < 100:
        bpm *= 2
    return bpm

def get_song_data(key):
    pack, song = key.split('~')
    folder = 'StepMania/Songs/{0}/{1}/'.format(pack, song)
    musicfiles = [file for file in listdir(folder) if file.endswith('.ogg') or file.endswith('.mp3')]
    music = folder + musicfiles[0]
    
    return [key, folder, music]

def get_song_features(key):
    song_data = get_song_data(key)
    if '{0}_beat_features.csv'.format(song_data[0]) in listdir('data'):
        print ('Song Already Loaded')
    else:
        SongFile(song_data[0], song_data[1], song_data[2], 'from_stepfile')

    gc.collect()

In [10]:
for song_name in songs_to_step:
    get_song_features('a_test~{0}'.format(song_name))

Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded
Song Already Loaded


In [11]:
samples_back_included = 8
num_classes = 5
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return (0, 0)
    return notes[index][0].count('1')
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key):
    X = []
    if '{0}_beat_features.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = len(beat_features)
        for i in range(num_notes):
            features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, i - j)]
            features.append(i % 48)
            features.append(get_beat_importance(i))
            features.append(i / 48)
            features.append(num_notes - i / 48)
            X.append(features)
    return np.array(X)

def calculate_importance(row):
    return (1 - row[0]) * (row[1] + row[2] * 2 + row[3] * 30 + row[4] * 40)

def step_song(key, clf):
    song_X = get_features_for_song(key)
    new_song_y = clf.predict(song_X)
    beat_importance = [calculate_importance(row) for row in new_song_y]
    
    pd.DataFrame(beat_importance).to_csv('generated_data/{0}_importance_generated.csv'.format(key), index=False)

In [13]:
beat_feature_model = load_model('models/beat_importance_model.h5')

In [14]:
for song_name in songs_to_step:
    step_song('a_test~{0}'.format(song_name), beat_feature_model)

In [15]:
steps_per_bar = 48
class WriteSongFile:
    def __init__(self, key, folder, music_file):
        misc = pd.read_csv('data/{0}_misc.csv'.format(key)).values
        self.beat_importance = pd.read_csv('generated_data/{0}_importance_generated.csv'.format(key), converters={'0': lambda x: float(x)}).values
        self.folder = folder
        self.name = key.split('~')[1]
        self.music_name = music_file
        self.offset = misc[0][0]
        self.beat_length = 60. / misc[1][0]
        self.bpm = misc[1][0]
        self.extension = music_file.split('.')[1]

songs = {}
for song_name in songs_to_step:
    key = 'a_test~{0}'.format(song_name)
    song_data = get_song_data(key)
    songs[key] = WriteSongFile(key, song_data[1], song_data[2])

In [16]:
beats_to_track = 48
num_classes = 3
class_map = {
    '0': 0,
    '1': 1,
    '2': 1,
    '3': 0,
    '4': 1,
    'M': 2
}

def get_is_note(i, notes):
    if i < 0:
        return [0, 0, 0, 0]
    return [char == '1' for char in notes[i][0]]

def get_is_mine(i, notes):
    if i < 0:
        return [0, 0, 0, 0]
    return [char == 'M' for char in notes[i][0]]

def get_note_class(note):
    if i < 0:
        return 0
    return class_map[note] if note in class_map else 0

def get_row_classes(row):
    return [get_note_class(note) for note in row[0]]

In [17]:
#TODO: build models for either side instead of one left one right
models = [load_model('models/no_filter_10_data_step_model_{0}.h5'.format(i)) for i in range(4)]

In [20]:
outputs = {
    0: '0',
    1: '1',
    2: 'M'
}
# TODO: make 3/4's less common by penalizing here or figure out how in model
def get_output_for_note(note_class_predictions):
    note_class_predictions[1] *= 1.5
    return outputs[np.argmax(note_class_predictions)]

def get_output_for_row(note_class_predictions):
    return ''.join([get_output_for_note(note[0]) for note in note_class_predictions])

def get_output(song):
    y = []
    length = len(song.beat_importance)
    is_note = [[0, 0, 0, 0]] * beats_to_track
    is_mine = [[0, 0, 0, 0]] * beats_to_track
    importances = [0 if i < 0 else song.beat_importance[i][0] for i in range(-beats_to_track, length)]
    for i in range(length):
        X_row = np.concatenate((np.array(is_note[-(beats_to_track - 1):]).flatten(), np.array(is_mine[-(beats_to_track - 1):]).flatten(), importances[i:i + beats_to_track]), axis=0)
        notes = [models[i].predict(np.array([X_row])) for i in range(4)]
        prediction = get_output_for_row(notes)
        is_note.append([char == '1' for char in prediction])
        is_mine.append([char == 'M' for char in prediction])
        y.append(prediction)

    return y

In [32]:
def write_song_header(output_stepfile, song):
    keys = ['TITLE', 'MUSIC', 'OFFSET', 'SAMPLESTART', 'SAMPLELENGTH', 'SELECTABLE', 'BPMS']
    header_info = {
        'TITLE': song.name,
        'MUSIC': '{0}.{1}'.format(song.name, song.extension),
        'OFFSET': -song.offset,
        'SAMPLESTART': song.offset + 32 * song.beat_length,
        'SAMPLELENGTH': 32 * song.beat_length,
        'SELECTABLE': 'YES',
        'BPMS': '0.000={:.3f}'.format(song.bpm)
    }
    
    for key in keys:
        print ("#{0}:{1};".format(key, str(header_info[key])), file=output_stepfile)
        
def write_step_header(output_stepfile, song):
    print("\n//---------------dance-single - J. Zukewich----------------", file=output_stepfile)
    print ("#NOTES:", file=output_stepfile)
    for detail in ['dance-single', 'J. Zukewich', 'Expert', '9', '0.242,0.312,0.204,0.000,0.000']:
        print ('\t{0}:'.format(detail), file=output_stepfile)
    
    for i in range(len(song.predicted_notes)):
        row = song.predicted_notes[i]
        print (row, file=output_stepfile)
        if i % steps_per_bar == steps_per_bar - 1:
            print (",", file=output_stepfile)

    print ("0000;", file=output_stepfile)
    
def step_song(song):
    stepfile_name = 'StepMania/Songs/a_test/{0}/{0}.sm'.format(song.name)
    output_stepfile=open(stepfile_name, 'w')
    write_song_header(output_stepfile, song)
    write_step_header(output_stepfile, song)
    output_stepfile.close()

In [33]:
for song_name in songs_to_step:
    song = songs['a_test~{0}'.format(song_name)]
    song.predicted_notes = get_output(song)
    step_song(song)